In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle
import os

def entrenar_y_guardar_modelos(datos_path, output_path="models/lstm_models.pkl"):
 """
 Entrena modelos LSTM para cada cliente y los guarda en un archivo pickle
 para uso posterior en la aplicación.
 """
 # Parámetros
 ventana = 24
 epochs = 30
 batch_size = 32
 
 # Cargar datos
 datos = pd.read_csv(datos_path, parse_dates=["Fecha"])
 
 # Diccionario para almacenar modelos por cliente
 modelos = {}
 
 # Ordenar por cliente y fecha
 datos = datos.sort_values(["Client_ID", "Fecha"])
 
 # Loop por cliente
 for cliente_id in datos["Client_ID"].unique():
     print(f"\nEntrenando modelo para {cliente_id}")
     
     df_cliente = datos[datos["Client_ID"] == cliente_id].copy()
     
     # Si hay pocos datos, omitir
     if len(df_cliente) < ventana + 1:
         print(f" - No hay suficientes datos para {cliente_id}. Se omite.")
         continue
     
     # Selección de variables y escalamiento
     variables = df_cliente[["Presion_actual", "Temperatura_actual", "Volumen_actual"]].values
     scaler = StandardScaler()
     variables_escaladas = scaler.fit_transform(variables)
     
     # Crear secuencias
     X, y = crear_secuencias(variables_escaladas, ventana)
     
     # División entrenamiento/prueba
     split = int(len(X) * 0.8)
     X_train, X_test = X[:split], X[split:]
     y_train, y_test = y[:split], y[split:]
     
     # Definir y entrenar modelo
     model = Sequential([
         LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False),
         Dense(X_train.shape[2])
     ])
     model.compile(optimizer='adam', loss='mse')
     
     model.fit(
         X_train, y_train,
         validation_split=0.1,
         epochs=epochs,
         batch_size=batch_size,
         verbose=0,
         callbacks=[EarlyStopping(patience=5, restore_best_weights=True)]
     )
     
     # Guardar el modelo y el escalador
     modelos[cliente_id] = (model, scaler)
 
 print(f"\n Modelos entrenados para {len(modelos)} clientes.")
 
 # Crear directorio si no existe
 os.makedirs(os.path.dirname(output_path), exist_ok=True)
 
 # Guardar modelos en archivo pickle
 with open(output_path, 'wb') as f:
     pickle.dump(modelos, f)
 
 print(f"Modelos guardados en {output_path}")
 
 return modelos

def crear_secuencias(datos, ventana):
 """Crea secuencias para entrenamiento LSTM"""
 X, y = [], []
 for i in range(len(datos) - ventana):
     X.append(datos[i:i+ventana])
     y.append(datos[i+ventana])
 return np.array(X), np.array(y)

if __name__ == "__main__":
 # Entrenar y guardar modelos
 entrenar_y_guardar_modelos("data/predicted_with_anomalies_and_future.csv")